In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math


In [2]:
stocks=pd.read_csv('S&p_500_stocks/sp_500_stocks.csv')

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data['symbol']

'AAPL'

In [5]:
price = data['latestPrice']
market_cap = data['marketCap']

In [6]:
price

143.82

In [7]:
my_columns = ['Ticker','stock Price','Market Capitialization','Number of shares to buy']

In [8]:
final_dataframe =pd.DataFrame(columns = my_columns)

In [9]:
final_dataframe.append(
    pd.Series([
        symbol,
        price,
        market_cap,
        'NA'
    ],index=my_columns),

    ignore_index = True
)

,Ticker,stock Price,Market Capitialization,Number of shares to buy
0,AAPL,143.82,2458024361949,NA


In [10]:
final_dataframe = pd.DataFrame(columns=my_columns)
for stock in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe=final_dataframe.append(pd.Series([stock,data['latestPrice'],data['marketCap'],'NA'],index = my_columns),ignore_index = True)

In [11]:
final_dataframe

,Ticker,stock Price,Market Capitialization,Number of shares to buy
0,A,179.940,54045974599,NA
1,AAL,20.079,13343577698,NA
2,AAP,212.450,13137185761,NA
3,AAPL,147.480,2390471201325,NA
4,ABBV,108.600,189815034908,NA
...,...,...,...,...
500,YUM,128.260,37953231478,NA
501,ZBH,147.400,30492858762,NA
502,ZBRA,560.270,30559400113,NA
503,ZION,56.120,9430025682,NA


In [12]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [13]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'], 
                                                   data[symbol]['quote']['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,stock Price,Market Capitialization,Number of shares to buy
0,A,178.100,53511238902,N/A
1,AAL,20.855,12960340138,N/A
2,AAP,214.910,13094219798,N/A
3,AAPL,144.100,2443987170764,N/A
4,ABBV,109.400,190908141976,N/A
...,...,...,...,...
500,YUM,126.140,37573020598,N/A
501,ZBH,150.300,30612413747,N/A
502,ZBRA,553.760,29800312992,N/A
503,ZION,55.990,9009378534,N/A


In [30]:
portfolio_size = input('Enter the value of your potfolio: ')

try:
    val =float(portfolio_size)
    print(val)
except ValueError:
    print('Thats not a number \n Please try again')
    portfolio_size= input('Enter the value of your portfolio: ')
    val=float(portfolio_size)

Enter the value of your potfolio: 10000000
10000000.0


In [39]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])-1):
    final_dataframe.loc[i, 'Number of shares to buy'] = math.floor(position_size / final_dataframe['stock Price'][i])
final_dataframe.drop('Number of shares to buy',axis=1,inplace=True)
final_dataframe

,Ticker,stock Price,Market Capitialization,Number Of Shares to Buy
0,A,178.100,53511238902,111.0
1,AAL,20.855,12960340138,949.0
2,AAP,214.910,13094219798,92.0
3,AAPL,144.100,2443987170764,137.0
4,ABBV,109.400,190908141976,181.0
...,...,...,...,...
500,YUM,126.140,37573020598,156.0
501,ZBH,150.300,30612413747,131.0
502,ZBRA,553.760,29800312992,35.0
503,ZION,55.990,9009378534,353.0


In [40]:
writer =pd.ExcelWriter('recommended trades.xlsx',engine = 'xlsxwriter')
final_dataframe.to_excel(writer,'Recommended Trades',index=False)

In [41]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [42]:
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_format #This applies the format 'string_format' to the column
                    )

0

In [43]:
# writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
# writer.sheets['Recommended Trades'].write('B1', 'Price', string_format)
# writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', string_format)
# writer.sheets['Recommended Trades'].write('D1', 'Number Of Shares to Buy', string_format)
# writer.sheets['Recommended Trades'].set_column('A:A', 20, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 20, integer_format)

In [44]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [45]:
writer.save()